In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/filtered-llm/filtered_df_test.csv
/kaggle/input/filtered-llm/filtered_df_val.csv
/kaggle/input/filtered-llm/filtered_df_train.csv


In [72]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, roc_auc_score
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer


In [73]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
train_df = pd.read_csv('/kaggle/input/filtered-llm/filtered_df_train.csv')
val_df = pd.read_csv('/kaggle/input/filtered-llm/filtered_df_val.csv')
test_df = pd.read_csv('/kaggle/input/filtered-llm/filtered_df_test.csv')

In [29]:
# Hyperparameters
max_vocab_size = 1000
max_sequence_length = 100
embedding_dim = 128
batch_size = 16
epochs = 50
num_classes = 5

In [74]:
def preprocess_text(texts):
    def clean_text(text):
        words = [word for word in text.split() if word not in stop_words]
        #  stemming
        stemmed_words = [stemmer.stem(word) for word in words]
        return ' '.join(stemmed_words)
    
    cleaned_texts = [clean_text(text) for text in texts]
    
    tokenizer = Tokenizer(num_words=max_vocab_size, lower=True, oov_token="<OOV>")
    tokenizer.fit_on_texts(cleaned_texts)
    sequences = tokenizer.texts_to_sequences(cleaned_texts)
    padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post')
    
    return padded_sequences, tokenizer


In [75]:
X_train = train_df['xi'] + " " + train_df['xj']
X_val = val_df['xi'] + " " + val_df['xj']

In [76]:
padded_sequences_train, tokenizer = preprocess_text(X_train)
padded_sequences_val, _ = preprocess_text(X_val)  

In [77]:
y_train = np.array(train_df['LLM_model'].factorize()[0])
y_val = np.array(val_df['LLM_model'].factorize()[0]) 


In [78]:
class TextDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.long)
        self.y = torch.tensor(y, dtype=torch.long)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [79]:
train_dataset = TextDataset(padded_sequences_train, y_train)
val_dataset = TextDataset(padded_sequences_val, y_val)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [80]:
class CNN_LSTM_Model(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes):
        super(CNN_LSTM_Model, self).__init__()
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        
        # Convolutional layer
        #self.conv1d = nn.Conv1d(embed_dim, 256, kernel_size=3, padding=1)  
        self.conv1d = nn.Conv1d(embed_dim, 128, kernel_size=3, padding=1)
        self.pool = nn.AdaptiveMaxPool1d(2)  
        
        # LSTM layer
        #self.lstm = nn.LSTM(256, 128, batch_first=True)  
        self.lstm = nn.LSTM(128, 64, batch_first=True,bidirectional=True)  
        
        # Fully connected layers
        #self.fc1 = nn.Linear(128, 64)  # Adjusted for LSTM output
        #self.dropout = nn.Dropout(0.3)  # Increased dropout
        #self.fc2 = nn.Linear(64, num_classes)
        
        self.fc1 = nn.Linear(64*2, 32)  
        self.dropout = nn.Dropout(0.4)  
        self.fc2 = nn.Linear(32, num_classes)

        # Batch Normalization
        self.batch_norm = nn.BatchNorm1d(128)  

    def forward(self, x):
        x = self.embedding(x).permute(0, 2, 1) 
        x = self.conv1d(x)
        x = self.batch_norm(x)  
        x = self.pool(x)  
        x = x.permute(0, 2, 1)  
        x, _ = self.lstm(x)  
        x = x[:, -1, :] 
        x = self.dropout(torch.relu(self.fc1(x)))  
        return self.fc2(x)  



In [81]:
model_cnn_lstm = CNN_LSTM_Model(vocab_size=max_vocab_size, embed_dim=embedding_dim, num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_cnn_lstm.parameters(), lr=5e-5,weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)

In [82]:
# Training Loop
def train_model(model, train_loader, val_loader, optimizer, criterion, epochs):
    for epoch in range(epochs):
        model.train()  
        total_loss, total_correct = 0, 0
        
        for X_batch, y_batch in tqdm(train_loader):
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            total_correct += (outputs.argmax(1) == y_batch).sum().item()
        
        accuracy = total_correct / len(train_loader.dataset)
        print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}, Accuracy: {accuracy}")

        # Validate
        val_loss, val_accuracy = validate_model(model, val_loader, criterion)
        print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_accuracy}")
        
        scheduler.step(val_loss)  

def validate_model(model, val_loader, criterion):
    model.eval()
    total_loss, total_correct = 0, 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            total_loss += loss.item()
            total_correct += (outputs.argmax(1) == y_batch).sum().item()
    
    val_accuracy = total_correct / len(val_loader.dataset)
    return total_loss / len(val_loader), val_accuracy


In [83]:
train_model(model_cnn_lstm, train_loader, val_loader, optimizer, criterion, epochs)

100%|██████████| 251/251 [00:01<00:00, 193.51it/s]


Epoch 1, Loss: 1.606983735741847, Accuracy: 0.20279720279720279
Validation Loss: 1.6115039363503456, Validation Accuracy: 0.194


100%|██████████| 251/251 [00:01<00:00, 203.62it/s]


Epoch 2, Loss: 1.5763034003664298, Accuracy: 0.2754745254745255
Validation Loss: 1.6218708343803883, Validation Accuracy: 0.182


100%|██████████| 251/251 [00:01<00:00, 203.10it/s]


...

Epoch 48, Loss: 1.0165874164892859, Accuracy: 0.5581918081918081
Validation Loss: 2.178909193724394, Validation Accuracy: 0.182


100%|██████████| 251/251 [00:01<00:00, 189.05it/s]


Epoch 49, Loss: 1.0202331519221879, Accuracy: 0.5526973026973027
Validation Loss: 2.1782063394784927, Validation Accuracy: 0.184


100%|██████████| 251/251 [00:01<00:00, 183.87it/s]

Epoch 50, Loss: 1.0176554426728968, Accuracy: 0.5467032967032966
Validation Loss: 2.175912868231535, Validation Accuracy: 0.18


In [84]:
import optuna

def objective(trial):
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    batch_size = trial.suggest_int('batch_size', 16, 128)
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
    lstm_hidden_units = trial.suggest_int('lstm_hidden_units', 64, 256)
    kernel_size = trial.suggest_int('kernel_size', 2, 5)
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-2)

    
    model_cnn_lstm = CNN_LSTM_Model(vocab_size=max_vocab_size, embed_dim=embedding_dim, num_classes=num_classes).to(device)
    model_cnn_lstm.dropout.p = dropout_rate  
    
    optimizer = optim.Adam(model_cnn_lstm.parameters(), lr=learning_rate)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    for epoch in range(10): 
        train_model(model_cnn_lstm, train_loader, val_loader, optimizer, criterion, epochs=1)
    
    # Validate
    val_loss, val_accuracy = validate_model(model_cnn_lstm, val_loader, criterion)
    
    return val_loss  


In [85]:
if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")  # Minimize validation loss
    study.optimize(objective, n_trials=50)  # Run 50 trials

    # Get the best hyperparameters
    best_params = study.best_params
    print("Best hyperparameters: ", best_params)

[I 2024-10-05 17:18:11,370] A new study created in memory with name: no-name-bca3c3a8-e3ad-412d-a7d9-0c023247bf93
/tmp/ipykernel_30/3963701134.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
/tmp/ipykernel_30/3963701134.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
/tmp/ipykernel_30/3963701134.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform('wei

Epoch 1, Loss: 1.6150681163583482, Accuracy: 0.2022977022977023
Validation Loss: 1.6139707139560155, Validation Accuracy: 0.204


100%|██████████| 112/112 [00:00<00:00, 187.10it/s]


Epoch 1, Loss: 1.6117220212306296, Accuracy: 0.21428571428571427
Validation Loss: 1.6112834811210632, Validation Accuracy: 0.192


100%|██████████| 112/112 [00:00<00:00, 191.74it/s]


...

Epoch 1, Loss: 0.7258738153321402, Accuracy: 0.6878121878121878
Validation Loss: 2.753805469382893, Validation Accuracy: 0.26


100%|██████████| 175/175 [00:01<00:00, 168.81it/s]


Epoch 1, Loss: 0.6424116609777724, Accuracy: 0.7345154845154845
Validation Loss: 2.910907057198611, Validation Accuracy: 0.242


100%|██████████| 175/175 [00:00<00:00, 191.05it/s]
[I 2024-10-05 17:21:52,502] Trial 49 finished with value: 3.1299676299095154 and parameters: {'learning_rate': 0.00010990050331830979, 'batch_size': 23, 'dropout_rate': 0.2554000123178178, 'lstm_hidden_units': 86, 'kernel_size': 3, 'weight_decay': 1.4773282094639088e-05}. Best is trial 40 with value: 1.594300888478756.


Epoch 1, Loss: 0.5721849296774183, Accuracy: 0.7462537462537463
Validation Loss: 3.1299676299095154, Validation Accuracy: 0.254
Best hyperparameters:  {'learning_rate': 1.620192336125112e-05, 'batch_size': 32, 'dropout_rate': 0.12942259933178754, 'lstm_hidden_units': 114, 'kernel_size': 4, 'weight_decay': 4.841313802691377e-06}


### Model with best hyperparams

In [86]:
class CNN_LSTM_Model(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes, lstm_hidden_units, kernel_size, dropout_rate):
        super(CNN_LSTM_Model, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.conv1d = nn.Conv1d(embed_dim, 128, kernel_size=kernel_size, padding=kernel_size // 2)  
        self.pool = nn.AdaptiveMaxPool1d(2)  
        self.lstm = nn.LSTM(128, lstm_hidden_units, batch_first=True, bidirectional=True) 
        self.fc1 = nn.Linear(lstm_hidden_units * 2, 32)  #  bidirectional LSTM 
        self.dropout = nn.Dropout(dropout_rate) 
        self.fc2 = nn.Linear(32, num_classes)
        self.batch_norm = nn.BatchNorm1d(128) 

    def forward(self, x):
        x = self.embedding(x).permute(0, 2, 1)  
        x = self.conv1d(x)
        x = self.batch_norm(x) 
        x = self.pool(x)  
        x = x.permute(0, 2, 1)  
        x, _ = self.lstm(x) 
        x = x[:, -1, :] 
        x = self.dropout(torch.relu(self.fc1(x)))  
        return self.fc2(x)  


In [87]:
best_hyperparameters = {
    'learning_rate': 1.620192336125112e-05,
    'batch_size': 32,
    'dropout_rate': 0.12942259933178754,
    'lstm_hidden_units': 114,
    'kernel_size': 4,
    'weight_decay': 4.841313802691377e-06
}

model_cnn_lstm_tuned = CNN_LSTM_Model(
    vocab_size=max_vocab_size, 
    embed_dim=embedding_dim, 
    num_classes=num_classes, 
    lstm_hidden_units=best_hyperparameters['lstm_hidden_units'],
    kernel_size=best_hyperparameters['kernel_size'],
    dropout_rate=best_hyperparameters['dropout_rate']
).to(device)


In [88]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(
    model_cnn_lstm_tuned.parameters(),  
    lr=best_hyperparameters['learning_rate'],  
    weight_decay=best_hyperparameters['weight_decay']  
)


In [89]:
train_model(model_cnn_lstm_tuned, train_loader, val_loader, optimizer, criterion, epochs)

100%|██████████| 251/251 [00:01<00:00, 195.04it/s]


Epoch 1, Loss: 1.606349453983079, Accuracy: 0.2315184815184815
Validation Loss: 1.608778677880764, Validation Accuracy: 0.204


100%|██████████| 251/251 [00:01<00:00, 185.89it/s]


Epoch 2, Loss: 1.597806261830121, Accuracy: 0.29120879120879123
Validation Loss: 1.6080564185976982, Validation Accuracy: 0.162


100%|██████████| 251/251 [00:01<00:00, 192.74it/s]


...

Epoch 48, Loss: 0.30809511998260164, Accuracy: 0.9000999000999002
Validation Loss: 4.002277918159962, Validation Accuracy: 0.188


100%|██████████| 251/251 [00:01<00:00, 201.16it/s]


Epoch 49, Loss: 0.29481900091546465, Accuracy: 0.9053446553446554
Validation Loss: 4.042600065469742, Validation Accuracy: 0.186


100%|██████████| 251/251 [00:01<00:00, 194.18it/s]

Epoch 50, Loss: 0.28373461563986135, Accuracy: 0.9090909090909091
Validation Loss: 4.14722765982151, Validation Accuracy: 0.202


In [90]:
def calculate_auc_roc(model, data_loader):
    model.eval()
    all_labels = []
    all_probs = []
    with torch.no_grad():
        for X_batch, y_batch in data_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            probs = torch.softmax(outputs, dim=1)
            all_probs.append(probs.cpu().numpy())
            all_labels.append(y_batch.cpu().numpy())
    
    all_labels = np.concatenate(all_labels)
    all_probs = np.concatenate(all_probs)
    return roc_auc_score(np.eye(num_classes)[all_labels], all_probs, multi_class='ovr')


In [91]:
val_auc_roc = calculate_auc_roc(model_cnn_lstm, val_loader)
print(f"Validation AUC-ROC: {val_auc_roc}")

Validation AUC-ROC: 0.49484500000000003


# Testing

In [92]:
X_test = test_df['xi'] + " " + test_df['xj']
padded_sequences_test, _ = preprocess_text(X_test)
y_test = np.array(test_df['LLM_model'].factorize()[0])

test_dataset = TextDataset(padded_sequences_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [93]:
test_loss, test_accuracy = validate_model(model_cnn_lstm, test_loader, criterion)
test_auc_roc = calculate_auc_roc(model_cnn_lstm, test_loader)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}, Test AUC-ROC: {test_auc_roc}")

Test Loss: 2.2013928703963757, Test Accuracy: 0.14770459081836326, Test AUC-ROC: 0.46646734648527205
